In [1]:
from tensorflow import keras
import numpy as np
import os
from seismic_purifier.config import BATCH_SIZE
from seismic_purifier.representation_learning_models import (
    RepresentationLearningSingleAutoencoder,
    RepresentationLearningDenoisingSingleAutoencoder,
    RepresentationLearningMultipleAutoencoder
)

2025-04-03 03:30:36.618070: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-03 03:30:36.862757: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-03 03:30:36.862805: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-03 03:30:36.864116: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-03 03:30:36.971957: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-03 03:30:36.973633: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [ ]:
# ============================
# 1. Configuration
# ============================

# Paths to your data
TRAIN_DATA_PATH = "/home/ege/Documents/STEAD_SUBSAMPLER/SLVR_NPY/KO.SLVT.2019-09-24T00:00:00.000000Z-2019-09-29T00:00:00.080000Z.npy"
TRAIN_LABEL_PATH = 'data/Y_train_1280sample.npy'  # Replace with your actual path
TEST_LABEL_PATH = 'data/Y_test_1280sample.npy'  # Replace with your actual path

# Directory to save checkpoints and the final model
CHECKPOINT_DIR = 'checkpoints'
MODEL_SAVE_PATH = 'checkpoints/SLVT_FULL_representation_cross_covariances_20epochs.h5'

# Training parameters
EPOCHS = 20
LEARNING_RATE = 1e-3

In [6]:
# Ensure directories exist
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)

# ============================
# 2. Data loading
# ============================
X_train = np.load(TRAIN_DATA_PATH)  # Expected shape: (num_samples, 3000, 3)
print(f"Training data shape: {X_train.shape}")

Training data shape: (14400, 3000, 3)


In [7]:
print(np.any(np.isnan(X_train)))
np.isnan(X_train).sum()

False


0

In [8]:
# ============================
# 3. Representation Learning Model Instantiation
# ============================

# Choose the model you want to train
# For example, using RepresentationLearningSingleAutoencoder
#model = RepresentationLearningSingleAutoencoder(
#    name="rep_learning_autoencoder"
#)

# Alternatively, you can choose other models:
# model = RepresentationLearningDenoisingSingleAutoencoder(
#     name="rep_learning_denoising_autoencoder",
#     input_noise_std=1e-6,
#     denoising_noise_std=2e-1
# )
model = RepresentationLearningMultipleAutoencoder(
     name="rep_learning_autoencoder_ensemble",
     input_noise_std=1e-6,
     eps=1e-27
)

In [9]:
model.name

'rep_learning_autoencoder_ensemble'

In [10]:
# ============================
# 4. Model Compilation
# ============================
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE) 
model.compile(optimizer=optimizer)

In [11]:
# ============================
# 5. Callbacks Setup
# ============================
# Define callbacks for saving checkpoints, early stopping.
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(CHECKPOINT_DIR, 'autoencoder_epoch_{epoch:02d}.h5'),
        save_weights_only=True,
        save_freq='epoch',
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=2,
        restore_best_weights=True,
        verbose=1
    )
]

In [12]:
# ============================
# 6. Training the Representation Learning Model and Save.
# ============================
fit_result = model.fit(X_train, 
                       epochs=EPOCHS, 
                       batch_size=BATCH_SIZE, 
                       callbacks=callbacks, 
                       shuffle=False)

model.save_weights(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")

Epoch 1/5
57/57 [==============================] - ETA: 0s - loss: 2.3096
Epoch 1: saving model to checkpoints/autoencoder_epoch_01.h5
57/57 [==============================] - 700s 9s/step - loss: 2.3096
Epoch 2/5
57/57 [==============================] - ETA: 0s - loss: 1.7966
Epoch 2: saving model to checkpoints/autoencoder_epoch_02.h5
57/57 [==============================] - 507s 9s/step - loss: 1.7966
Epoch 3/5
57/57 [==============================] - ETA: 0s - loss: 1.5062
Epoch 3: saving model to checkpoints/autoencoder_epoch_03.h5
57/57 [==============================] - 492s 9s/step - loss: 1.5062
Epoch 4/5
57/57 [==============================] - ETA: 0s - loss: 1.3458
Epoch 4: saving model to checkpoints/autoencoder_epoch_04.h5
57/57 [==============================] - 486s 9s/step - loss: 1.3458
Epoch 5/5
57/57 [==============================] - ETA: 0s - loss: 1.2501
Epoch 5: saving model to checkpoints/autoencoder_epoch_05.h5
57/57 [==============================] - 494s 9s/